<a href="https://colab.research.google.com/github/jh941213/crawling/blob/main/05_selenium%E1%84%8B%E1%85%B3%E1%86%AF_%E1%84%8B%E1%85%B5%E1%84%8B%E1%85%AD%E1%86%BC%E1%84%92%E1%85%A1%E1%86%AB_%E1%84%83%E1%85%A9%E1%86%BC%E1%84%8C%E1%85%A5%E1%86%A8%E1%84%8F%E1%85%B3%E1%84%85%E1%85%A9%E1%86%AF%E1%84%85%E1%85%B5%E1%86%BC(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 국민청원 데이터 가져오기

셀레니움 , 그는 웹테스팅을 위해 만들어진 존재
* 강점
1) 브라우저를 직접 실행하는 기능
-> 브라우저로 접근할 수 있는 페이지는 다 접근 가능하다

2) 브라우저에서 마우스와 키보드 컨트롤 기능
-> 마우스로 클릭할 수 있다 
	-> 해당 태그를 파싱할 수 있다
	-> 페이지의 이동이 자유롭다

* 약점
- 웹드라이버 프로그램을 설치해야 한다.
- 불안정하다.
- 느리다.


* 사전지식
- selenium에서는 tag를 element라고 명명한다.
- selenium에서는 원하는 element를 xpath를 이용해 찾으면 된다.
    -> 수식어 == xpath(그 외는 무시) ; == 족보+속성


- 참고:
  - [국민청원 데이터셋 CSV](https://newhiwoong.github.io/%EA%B5%AD%EB%AF%BC%EC%B2%AD%EC%9B%90/%EA%B5%AD%EB%AF%BC%EC%B2%AD%EC%9B%90-%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%85%8B)
  - [청와대 국민청원 https://www1.president.go.kr/petitions](https://www1.president.go.kr/petitions)
  - [Using Selenium with Google Colaboratory https://darektidwell.com/using-selenium-with-google-colaboratory/](https://darektidwell.com/using-selenium-with-google-colaboratory/)
  - [https://stackoverflow.com/questions/56829470/selenium-google-colab-error-chromedriver-executable-needs-to-be-in-path](https://stackoverflow.com/questions/56829470/selenium-google-colab-error-chromedriver-executable-needs-to-be-in-path)
 
- 참고링크 중 "국민청원 데이터셋 CSV"의 내용을 토대로 수정하여 작성
- 1건당 1-2초(google colab에서 더 느림, 실 사용은 로컬을 권장함)

## 확인된 에러의 경우 
1. 없는 페이지
2. 관리자 차단 페이지

In [ ]:
//*[@id="content"]/table/tbody/tr/td[3]/div/strong
//*[@id="content"]/table/tbody/tr[4]/td[3]/div/strong

In [ ]:
# install chromium, its driver, and selenium
if 'google.colab' in sys.modules:
    !apt-get update
    !apt install chromium-chromedriver
    !cp /usr/lib/chromium-browser/chromedriver /usr/bin
    !pip install selenium

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 https://cloud.r-project.org/bin/linux/ubuntu bi

In [ ]:
import sys

# set options to be headless, ..
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup
import re

if 'ipykernel' in sys.modules:
    from tqdm.notebook import tqdm
else:
    from tqdm import tqdm
    
import pandas as pd

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

In [ ]:
petition_info = {'pet_id':[], 'progress':[], 'title':[],  'like':[],
                'contents':[], 'answer':[], 'category':[],
                'sday':[], 'eday':[], 'proponent':[]}

In [ ]:
rtn = []
wd = webdriver.Chrome('chromedriver',options=options)

pet_id = 604660

url='https://www1.president.go.kr/petitions/' + str(pet_id)
wd.get(url) # 비교 : resp = requests.get(url)
element = WebDriverWait(wd, 3).until(EC.presence_of_element_located((By.CLASS_NAME, 'petitionsView_title')))
# time.sleep(3) # 명시적 대기 -> 쓰지 말기

# select => find_element_by_xpath, find_elements_by_xpath
progress = wd.find_element_by_xpath('//*[@id="cont_view"]/div[2]/div[1]/div/div[1]/div/div[1]/h4').text
title = wd.find_element_by_xpath('//*[@id="cont_view"]/div[2]/div[1]/div/div[1]/div/h3').text
like = wd.find_element_by_xpath('//*[@id="cont_view"]/div[2]/div[1]/div/div[1]/div/h2/span').text

if progress == '답변완료':
    contents = wd.find_element_by_xpath('//*[@id="cont_view"]/div[2]/div[1]/div/div[1]/div/div[4]/div[4]').text

    answer = wd.find_element_by_xpath('//*[@id="cont_view"]/div[2]/div[1]/div/div[1]/div/div[5]/div').text
else:
    contents = wd.find_element_by_xpath('//*[@id="cont_view"]/div[2]/div[1]/div/div[1]/div/div[4]/div[2]').text

    answer = ''

info = wd.find_element_by_xpath('//*[@id="cont_view"]/div[2]/div[1]/div/div[1]/div/div[2]').text.split('\n')
category = info[1]
sday = info[3]
eday = info[5]
proponent = info[7]

info_list = [pet_id, progress, title, like, contents, answer, category, sday, eday, proponent]

for k, i in zip(petition_info.keys(), info_list):
    petition_info[k].append(i)
petition_info

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead


pet_id 604660
progress 청원진행중
title 윤석열 당선자의 대장동 부산저축은행 부실수사 봐주기 의혹과 김건희의 주가조작 실체의 진상조사 확인을 위한 청원입니다.
like 521,315
contents 20대 대통령으로 윤석열 후보가 당선되었습니다.

그러나 여전히 본인과 그부인에 대한 의혹은 좀처럼 사라지지 않고 있습니다.

대통령 후보라면 그 친가족 검증에 있어서는 먼지털 하나라도 검증대상입니다.

여전히 의혹만 있을뿐 이에 대한 검증과 수사가 제대로 이루어지지 않은 상태에서 당선인 신분이 된 것은 전혀

납득이 안될 일입니다.

윤석열 당선인이 진정 국민과 나라를 위해 일할 지도자가 된다면 최소 이정도는 당선인 신분에도 확실한 검증이

필요합니다.

이것이 과연 윤석열 당선인이 외쳤던 공정과 상식입니까?

도저히 국민의 한사람으로써 납득이 안될 일입니다.

물론 대장동 특검에 있어서는 여당에서 대선후에도 강력추진하겠다란 의지는 확고합니다.

그러나 여전히 이부분에 대해서 윤석열 당선인은 제대로된 답변과 언급이 없는 상태로 그냥 당선되면 없어진다.

하는 스스로의 망각된 생각을 가진 자가 20대 대통령이 된다는거 국민의 한사람으로서 도저히 납득이 안되고 이해

불가입니다.

윤석열 당선인의 부산저축은행 부실수사 의혹과 김건희씨의 주가조작 의혹에 대해 당선인 신분에 있어서도 강력한

수사가 이루어져야 할 것이며, 필요시 이에대한 법적제재를 받는 것, 특히나 당선자 역시국민중 한사람이라는 명분

잊어서는 안될 것입니다.

안될 것입니다.
answer 
category 정치개혁
sday 2022-03-11
eday 2022-04-10
proponent kakao - ***


{'answer': ['', ''],
 'category': ['정치개혁', '정치개혁'],
 'contents': ['20대 대통령으로 윤석열 후보가 당선되었습니다.\n\n그러나 여전히 본인과 그부인에 대한 의혹은 좀처럼 사라지지 않고 있습니다.\n\n대통령 후보라면 그 친가족 검증에 있어서는 먼지털 하나라도 검증대상입니다.\n\n여전히 의혹만 있을뿐 이에 대한 검증과 수사가 제대로 이루어지지 않은 상태에서 당선인 신분이 된 것은 전혀\n\n납득이 안될 일입니다.\n\n윤석열 당선인이 진정 국민과 나라를 위해 일할 지도자가 된다면 최소 이정도는 당선인 신분에도 확실한 검증이\n\n필요합니다.\n\n이것이 과연 윤석열 당선인이 외쳤던 공정과 상식입니까?\n\n도저히 국민의 한사람으로써 납득이 안될 일입니다.\n\n물론 대장동 특검에 있어서는 여당에서 대선후에도 강력추진하겠다란 의지는 확고합니다.\n\n그러나 여전히 이부분에 대해서 윤석열 당선인은 제대로된 답변과 언급이 없는 상태로 그냥 당선되면 없어진다.\n\n하는 스스로의 망각된 생각을 가진 자가 20대 대통령이 된다는거 국민의 한사람으로서 도저히 납득이 안되고 이해\n\n불가입니다.\n\n윤석열 당선인의 부산저축은행 부실수사 의혹과 김건희씨의 주가조작 의혹에 대해 당선인 신분에 있어서도 강력한\n\n수사가 이루어져야 할 것이며, 필요시 이에대한 법적제재를 받는 것, 특히나 당선자 역시국민중 한사람이라는 명분\n\n잊어서는 안될 것입니다.\n\n안될 것입니다.',
  '20대 대통령으로 윤석열 후보가 당선되었습니다.\n\n그러나 여전히 본인과 그부인에 대한 의혹은 좀처럼 사라지지 않고 있습니다.\n\n대통령 후보라면 그 친가족 검증에 있어서는 먼지털 하나라도 검증대상입니다.\n\n여전히 의혹만 있을뿐 이에 대한 검증과 수사가 제대로 이루어지지 않은 상태에서 당선인 신분이 된 것은 전혀\n\n납득이 안될 일입니다.\n\n윤석열 당선인이 진정 국민과 나라를 위해 일할 지도자가 된다면 최소 이정도는 당선인 신분에